In [95]:
import requests
from pprint import pprint
import pandas as pd
import json
from gensim import corpora, models, similarities

In [96]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [97]:
url = ("http://localhost:8082/api/populatedTags");
response = requests.get(url)
data = json.loads(response.text)
pprint(data)

[{'__v': 0,
  '_chunks': [{'_id': '56aba31b9b1c17c8c8532b0d',
               'ad_id': 2597,
               'chunk_id': 53704,
               'content': 'The Public Sector Health team is searching for a '
                          'sales executive to biggest Healthcare customers in '
                          'Finland. The Account Executive (Healthcare) role '
                          'adds value to Microsoft by delivering a '
                          'well-managed, profitable and growing business '
                          'produced through relationship- and sales-excellence '
                          'practices where the customer views the Account '
                          'Executive as a trusted advisor. Vertical '
                          'understanding of Healthcare and solutions. Sales '
                          'Excellence in Relationship Management, Opportunity '
                          'Management and Business Management enabled through '
                          'to

In [98]:
class TagCorpus(object):
    def __iter__(self):
        for tag in data:
            words = ""
            for chunk in tag['_chunks']:
                words += " " + str(chunk['content']);
            yield words.split()

In [99]:
corpus = TagCorpus() # doesn't load the corpus into memory!
print(corpus)

In [100]:
for vector in corpus: # load one vector into memory at a time
    print(vector)

['The', 'Public', 'Sector', 'Health', 'team', 'is', 'searching', 'for', 'a', 'sales', 'executive', 'to', 'biggest', 'Healthcare', 'customers', 'in', 'Finland.', 'The', 'Account', 'Executive', '(Healthcare)', 'role', 'adds', 'value', 'to', 'Microsoft', 'by', 'delivering', 'a', 'well-managed,', 'profitable', 'and', 'growing', 'business', 'produced', 'through', 'relationship-', 'and', 'sales-excellence', 'practices', 'where', 'the', 'customer', 'views', 'the', 'Account', 'Executive', 'as', 'a', 'trusted', 'advisor.', 'Vertical', 'understanding', 'of', 'Healthcare', 'and', 'solutions.', 'Sales', 'Excellence', 'in', 'Relationship', 'Management,', 'Opportunity', 'Management', 'and', 'Business', 'Management', 'enabled', 'through', 'tools,', 'processes', 'and', 'scorecards.', 'Together,', 'they', 'build', 'strong', 'trusted', 'advisor', 'relationships,', 'yield', 'high', 'customer', 'satisfaction', 'and', 'goal', 'attainment.']
['Account', 'Executive', 'Health', '(902280)']
['As', 'a', 'Accoun

In [101]:
stoplist = set('for a of the and to in'.split())
# collect statistics about all tokens
dictionary = corpora.Dictionary(vector for vector in corpus)
# remove stop words and words that appear only once
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist
            if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.items() if docfreq == 1]

# dictionary.filter_tokens(stop_ids + once_ids) # remove stop words and words that appear only once
dictionary.filter_tokens(stop_ids)

dictionary.compactify() # remove gaps in id sequence after words that were removed
print(dictionary)

Dictionary(179 unique tokens: ['opportunity', 'key', 'opportunities', 'high', 'well-managed,']...)
